In [4]:
import numpy as np
import pandas as pd
import faiss
import time
from sklearn.cluster import KMeans
from collections import defaultdict

In [20]:
# 这个应该是未经训练的roberta直接输出的结果
res = np.load('./res.npy')

In [21]:
s = res.reshape(1786279,1024)

## sklearn
非常慢

In [ ]:
kmeans = KMeans(n_clusters=135,random_state=0).fit(s)
center = kmeans.cluster_centers_
np.save('./data/bert_center.npy',center)

## faiss 
又快得害怕

In [4]:
n_center = 135
niter = 100
d=1024

In [13]:
fkmeans = faiss.Kmeans(d,n_center,niter=niter,verbose = True)

TypeError: in method 'ClusteringParameters_verbose_set', argument 2 of type 'bool'

In [14]:
%%time
fkmeans.train(s)

CPU times: user 1min 15s, sys: 766 ms, total: 1min 16s
Wall time: 4.78 s


44125.15625

In [2]:
kaggle = pd.read_json('./kaggle_data/arxiv-metadata-oai-snapshot.json',lines=True)
#kaggle.head()

In [3]:
label = kaggle['categories'].apply(lambda x : set(x.split()))
label = label.to_list()

In [ ]:
#找据数据最近的一个中心，也就是属于那个聚类
D, I = fkmeans.search(s,1)

In [22]:
index = faiss.IndexFlatL2 (1024)
index.add(s)

In [65]:
D, I = index.search (fkmeans.centroids, 100)

In [23]:
#只找一个的话，应该是它本身了吧
D, I = index.search (fkmeans.centroids, 1)

In [6]:
#np.save('./kaggle_data/tmp.npy',I)
I = np.load('./kaggle_data/tmp.npy')

In [12]:
I = list(I)

In [16]:
I[0]

array([1520554])

In [17]:
label[int(I[i])]

{'cond-mat.dis-nn'}

In [22]:
dic = defaultdict(list)
for i in range(len(I)):
    dic[','.join(list(label[int(I[i])]))].append(int(I[i]))

In [24]:
len(dic)

92

In [25]:
kaggle.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [31]:
'https://arxiv.org/abs/'+kaggle.loc[1676474,'id']

'https://arxiv.org/abs/math/0009176'

In [33]:
'cs.AI' in dic

True

In [23]:
dic

defaultdict(list,
            {'cond-mat.dis-nn': [1520554],
             'cond-mat.str-el,cond-mat.quant-gas': [742692],
             'math.AP': [778553],
             'cond-mat.str-el': [94806, 1500056, 273363, 1155834],
             'math.MP,math-ph,math.NA,math.OC': [903617],
             'cond-mat.supr-con': [422771, 640102, 1531107],
             'nucl-th': [1742978],
             'math.MP,math-ph,math.DS': [1670550],
             'hep-lat': [386693],
             'math.SG,math.DS': [1676474, 1063356],
             'math.CA': [795859, 809571, 431535],
             'astro-ph.HE,nucl-th': [711051],
             'stat.CO': [702885],
             'cs.NA,math.NA': [845209],
             'math.GR,math.CO': [150894],
             'cond-mat.quant-gas': [461962],
             'math.MP,math-ph,nlin.SI,hep-th': [1225337],
             'cs.CV': [661109, 1366337],
             'cs.CR,cs.LG,stat.ML': [1202268],
             'physics.optics,quant-ph,cond-mat.stat-mech': [1348011],
             

## 实验结果
下一步作什么实验，需要把数据集分成每条一个样本，再跑一次吗?

In [113]:
ans = [0] * 135
for i in range(len(fcenters)):
    center = I[i][0]
    for j in range(1,100):
        index = I[i][j]
        if len(label[index] & label[center]) > 0:
            ans[i] += 1

In [115]:
dic = {}
for i in range(len(fcenters)):
    center = I[i][0]
    for j in range(1,100):
        index = I[i][j]
        if len(label[index] & label[center]) > 0:
            dic[','.join(list(label[center]))] =dic.get(','.join(list(label[center])),0)+ 1

In [112]:
#小于135因为有些中心的标签是一样的，也许说明了分类不好
len(dic)

91

In [116]:
dic

{'cond-mat.dis-nn': 5,
 'cond-mat.str-el,cond-mat.quant-gas': 29,
 'math.AP': 5,
 'cond-mat.str-el': 41,
 'math-ph,math.OC,math.NA,math.MP': 16,
 'cond-mat.supr-con': 22,
 'nucl-th': 24,
 'math-ph,math.DS,math.MP': 10,
 'hep-lat': 12,
 'math.SG,math.DS': 11,
 'math.CA': 17,
 'nucl-th,astro-ph.HE': 11,
 'stat.CO': 5,
 'math.CO,math.GR': 7,
 'cond-mat.quant-gas': 15,
 'math-ph,hep-th,math.MP,nlin.SI': 31,
 'cs.CV': 65,
 'cs.LG,stat.ML,cs.CR': 34,
 'cond-mat.stat-mech,physics.optics,quant-ph': 23,
 'hep-th,math.DG,math.CV': 16,
 'math-ph,physics.plasm-ph,math.MP': 17,
 'hep-th,hep-lat,gr-qc,cond-mat.other': 24,
 'astro-ph.SR': 28,
 'math.DG': 18,
 'hep-th,gr-qc': 64,
 'math.NT': 21,
 'cond-mat.mes-hall': 131,
 'astro-ph.GA': 54,
 'math.FA': 30,
 'cond-mat.stat-mech': 7,
 'astro-ph.SR,astro-ph.HE': 24,
 'math.PR,math.CV': 12,
 'cond-mat.mes-hall,quant-ph': 31,
 'math.QA,math.RT': 5,
 'physics.comp-ph,gr-qc': 2,
 'math.NA': 30,
 'cs.SI,cs.IR': 9,
 'math.CO': 27,
 'hep-th': 175,
 'cs.LG,cs.R

In [106]:
label[1520554]

{'cond-mat.dis-nn'}

In [93]:
data = kaggle.join(kaggle['categories'].str.split(' ', expand=True).stack().reset_index(level=1, drop=True).rename('label'))

In [96]:
from collections import Counter
num = Counter(data['label'])
num['cond-mat.dis-nn']

19010

In [97]:
num

Counter({'hep-ph': 152987,
         'math.CO': 47291,
         'cs.CG': 4760,
         'physics.gen-ph': 9340,
         'math.CA': 17299,
         'math.FA': 23023,
         'cond-mat.mes-hall': 70585,
         'gr-qc': 84540,
         'cond-mat.mtrl-sci': 67236,
         'astro-ph': 105380,
         'math.NT': 30644,
         'math.AG': 41249,
         'math.AT': 11715,
         'hep-th': 140061,
         'math.PR': 41387,
         'hep-ex': 42712,
         'nlin.PS': 7728,
         'physics.chem-ph': 14737,
         'q-bio.MN': 3048,
         'math.NA': 22610,
         'cond-mat.str-el': 58397,
         'cond-mat.stat-mech': 60019,
         'math.RA': 13284,
         'physics.optics': 31304,
         'physics.comp-ph': 15225,
         'q-bio.PE': 8568,
         'q-bio.CB': 1752,
         'quant-ph': 107734,
         'q-bio.QM': 6722,
         'hep-lat': 23815,
         'nucl-th': 48316,
         'math.OA': 10183,
         'math.QA': 16101,
         'math-ph': 63304,
         'math.MP